In [1]:
from helper_loader import *

In [2]:
original_normal = np.array([-1, 0, 0])
original_arrow = vedo.Arrow(end_pt=original_normal, c="blue", s=0.002)

pitch = 40
yaw = 60

vedo_rotated_normal = compute_normal(
    Orientation.HORIZONTAL, pitch, yaw, pitch_first=True
)
vedo_rotated_arrow = vedo.Arrow(end_pt=vedo_rotated_normal, c="black", s=0.002)

rotation = scipy.spatial.transform.Rotation.from_rotvec([0, yaw, pitch], degrees=True)
scipy_rotated_normal = rotation.apply(original_normal)
scipy_rotated_arrow = vedo.Arrow(end_pt=scipy_rotated_normal, c="white", s=0.002)

custom_rotated_normal = (
    np.array(
        [
            [math.cos(math.radians(yaw)), 0, math.sin(math.radians(yaw))],
            [0, 1, 0],
            [-math.sin(math.radians(yaw)), 0, math.cos(math.radians(yaw))],
        ]
    )
    @ np.array(
        [
            [math.cos(math.radians(pitch)), -math.sin(math.radians(pitch)), 0],
            [math.sin(math.radians(pitch)), math.cos(math.radians(pitch)), 0],
            [0, 0, 1],
        ]
    )
    @ original_normal
)
# custom_rotated_normal[1] *= -1
custom_rotated_arrow = vedo.Arrow(end_pt=custom_rotated_normal, c="purple", s=0.002)

print(f"Vedo rotated normal:   {format_list(vedo_rotated_normal)}")
print(f"Scipy rotated normal:  {format_list(scipy_rotated_normal)}")
print(
    f"Custom rotated normal: {format_list(custom_rotated_normal)}",
    flush=True,
)

show(
    [
        original_arrow,
        vedo_rotated_arrow,
        scipy_rotated_arrow,
        custom_rotated_arrow,
        *axes,
    ],
    camera=coronal_camera,
)

Vedo rotated normal:   -0.643 |  0.383 |  0.663
Scipy rotated normal:  -0.307 | -0.528 |  0.792
Custom rotated normal: -0.383 | -0.643 |  0.663


In [3]:

            pitch_rotation = np.array(
                [
                    [-math.cos(pitch), math.sin(pitch), 0],
                    [math.sin(pitch), math.cos(pitch), 0],
                    [0, 0, 1],
                ]
            )
            yaw_rotation = np.array(
                [
                    [math.cos(yaw), 0, math.sin(yaw)],
                    [0, 1, 0],
                    [-math.sin(yaw), 0, math.cos(yaw)],
                ]
            )

In [4]:
from itertools import permutations

list(permutations(["A", "B", "C"]))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

In [5]:
from itertools import permutations


def apply_rotation(
    vector: np.ndarray,
    orientation: Orientation,
    pitch: int = 0,
    yaw: int = 0,
    pitch_first: bool = True,
) -> np.ndarray:
    pitch = math.radians(pitch)
    yaw = math.radians(yaw)

    match orientation:
        case Orientation.CORONAL:
            pitch_rotation = np.array(
                [
                    [math.cos(pitch), -math.sin(pitch), 0],
                    [math.sin(pitch), math.cos(pitch), 0],
                    [0, 0, 1],
                ]
            )
            yaw_rotation = np.array(
                [
                    [math.cos(yaw), 0, math.sin(yaw)],
                    [0, 1, 0],
                    [-math.sin(yaw), 0, math.cos(yaw)],
                ]
            )
        case Orientation.HORIZONTAL:
            pitch_rotation = np.array(
                [
                    [math.cos(pitch), -math.sin(pitch), 0],
                    [math.sin(pitch), math.cos(pitch), 0],
                    [0, 0, 1],
                ]
            )
            yaw_rotation = np.array(
                [
                    [1, 0, 0],
                    [0, math.cos(yaw), -math.sin(yaw)],
                    [0, math.sin(yaw), math.cos(yaw)],
                ]
            )
        case Orientation.SAGITTAL:
            pitch_rotation = np.array(
                [
                    [1, 0, 0],
                    [0, math.cos(pitch), -math.sin(pitch)],
                    [0, math.sin(pitch), math.cos(pitch)],
                ]
            )
            yaw_rotation = np.array(
                [
                    [math.cos(yaw), 0, math.sin(yaw)],
                    [0, 1, 0],
                    [-math.sin(yaw), 0, math.cos(yaw)],
                ]
            )

    if pitch_first:
        rotated_vector = yaw_rotation @ pitch_rotation @ vector
    else:
        rotated_vector = pitch_rotation @ yaw_rotation @ vector

    return rotated_vector

In [6]:
def permutations_checker(vector, orientation, pitch, yaw):
    pitch = math.radians(pitch)
    yaw = math.radians(yaw)

    if orientation == Orientation.CORONAL:
        a = [math.cos(pitch), -math.sin(pitch), 0]
        b = [math.sin(pitch), math.cos(pitch), 0]
        c = [0, 0, 1]

        A = "[math.cos(pitch), -math.sin(pitch), 0]"
        B = "[math.sin(pitch), math.cos(pitch), 0]"
        C = "[0, 0, 1]"

        d = [math.cos(yaw), 0, math.sin(yaw)]
        e = [0, 1, 0]
        f = [-math.sin(yaw), 0, math.cos(yaw)]

        D = "[math.cos(yaw), 0, math.sin(yaw)]"
        E = "[0, 1, 0]"
        F = "[-math.sin(yaw), 0, math.cos(yaw)]"
    elif orientation == Orientation.HORIZONTAL:
        a = [math.cos(pitch), -math.sin(pitch), 0]
        b = [math.sin(pitch), math.cos(pitch), 0]
        c = [0, 0, 1]

        A = "[math.cos(pitch), -math.sin(pitch), 0]"
        B = "[math.sin(pitch), math.cos(pitch), 0]"
        C = "[0, 0, 1]"

        d = [1, 0, 0]
        e = [0, math.cos(yaw), -math.sin(yaw)]
        f = [0, math.sin(yaw), math.cos(yaw)]

        D = "[1, 0, 0]"
        E = "[0, math.cos(yaw), -math.sin(yaw)]"
        F = "[0, math.sin(yaw), math.cos(yaw)]"
    elif orientation == Orientation.SAGITTAL:
        a = [1, 0, 0]
        b = [0, math.cos(pitch), -math.sin(pitch)]
        c = [0, math.sin(pitch), math.cos(pitch)]

        A = "[1, 0, 0]"
        B = "[0, math.cos(pitch), -math.sin(pitch)]"
        C = "[0, math.sin(pitch), math.cos(pitch)]"

        d = [math.cos(yaw), 0, math.sin(yaw)]
        e = [0, 1, 0]
        f = [-math.sin(yaw), 0, math.cos(yaw)]

        D = "[math.cos(yaw), 0, math.sin(yaw)]"
        E = "[0, 1, 0]"
        F = "[-math.sin(yaw), 0, math.cos(yaw)]"

    combinations = []

    pitch_permutations = list(permutations([a, b, c]))
    pitch_strings = list(permutations([A, B, C]))
    yaw_permutations = list(permutations([d, e, f]))
    yaw_strings = list(permutations([D, E, F]))

    for i in range(len(pitch_permutations)):
        pitch_permutation = pitch_permutations[i]
        pitch_string = pitch_strings[i]

        pitch_rotation = np.array([*pitch_permutation])

        for i in range(len(yaw_permutations)):
            yaw_permutation = yaw_permutations[i]
            yaw_string = yaw_strings[i]

            yaw_rotation = np.array([*yaw_permutation])

            combinations.append(
                [
                    yaw_rotation @ pitch_rotation @ vector,
                    pitch_string,
                    yaw_string,
                ]
            )

    return combinations


orientation = Orientation.CORONAL
original_normal = np.array([-1, 0, 0])
pitch = 56
yaw = 32

combinations = permutations_checker(
    original_normal,
    orientation,
    pitch,
    yaw,
)

vedo_rotated_normal = compute_normal(
    orientation,
    pitch,
    yaw,
    pitch_first=True,
)
print("Expected: ", format_list(vedo_rotated_normal))

for result, pitch_string, yaw_string in combinations:
    # print(result)
    if not np.allclose(np.array(result), vedo_rotated_normal):
        continue

    print(result)
    print(pitch_string)
    print(yaw_string)

show(
    [
        vedo.Arrow(end_pt=vedo_rotated_normal, c="black", s=0.002),
        *[
            vedo.Arrow(end_pt=combination[0] * 2, c="white", s=0.002)
            for combination in combinations
        ],
        *axes,
    ],
    camera=sagittal_camera,
)

Expected:  -0.474 | -0.829 |  0.296
[-0.47422248 -0.82903757  0.29632709]
('[math.cos(pitch), -math.sin(pitch), 0]', '[math.sin(pitch), math.cos(pitch), 0]', '[0, 0, 1]')
('[math.cos(yaw), 0, math.sin(yaw)]', '[0, 1, 0]', '[-math.sin(yaw), 0, math.cos(yaw)]')


In [7]:
orientation = Orientation.SAGITTAL
normal = np.array([0, 0, 1])
pitch = 70
yaw = 20

vedo_normal = compute_normal(orientation, pitch, yaw)
custom_normal = apply_rotation(normal, orientation, pitch, yaw)

pitch_rotation = scipy.spatial.transform.Rotation.from_rotvec(
    [pitch, 0, 0], degrees=True
)
yaw_rotation = scipy.spatial.transform.Rotation.from_rotvec([0, yaw, 0], degrees=True)
scipy_normal = yaw_rotation.apply(pitch_rotation.apply(normal))

print(f"Baseline normal: {format_list(vedo_normal)}")
print(f"Custom normal:   {format_list(custom_normal)}")
print(f"Scipy normal:    {format_list(scipy_normal)}")

Baseline normal:  0.117 | -0.940 |  0.321
Custom normal:    0.117 | -0.940 |  0.321
Scipy normal:     0.117 | -0.940 |  0.321
